In [1]:
import numpy as np
import pandas as pd

In [2]:
flights = pd.read_csv('flights.csv')
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00


1. How many instances (observations) are in the dataset?

In [23]:
flights.shape[0]

336776

2. Missing values questions:
    a. Which two variables have the most missing values?
    b. How many missing values are there for these variables?

In [22]:
flights.isnull().sum().nlargest(2)

arr_delay    9430
air_time     9430
dtype: int64

3. What is the average (non-missing) air_time for the flights in the dataset?

In [21]:
flights['air_time'].mean() # automatically ignores missing values

150.68646019807787

4. How many unique dest values (destinations) are in the dataset?

In [20]:
flights['dest'].nunique()

105

5. What are the five most common destinations for the flights in the dataset (in order)?
    a. (most common)
    b.
    c.
    d.
    e. (fifth most common)

In [19]:
flights['dest'].value_counts().head(5)

dest
ORD    17283
ATL    17215
LAX    16174
BOS    15508
MCO    14082
Name: count, dtype: int64

6. What is the maximum distance (in miles) flown in the dataset?

In [7]:
flights['distance'].max()

4983

7. How many flights in the dataset flew the maximum distance?

In [17]:
(flights['distance'] == 4983).sum()

342

8. Create a variable that indicates whether or not a flight was cancelled based on whether or not the air_time variable is missing.
    a. What month has the highest proportion of cancelled flights? (Use the two digit month number - for example, if the answer is January, use "01" in the answer file)
    b. What is the proportion from (a.)?
    c. What origin has the lowest proportion of cancelled flights?
    d. What is the proportion from (c.)?

In [27]:
flights['cancelled'] = flights['air_time'].isnull()
monthly_flight_cancellations = flights.groupby('month')['cancelled'].mean()
print(monthly_flight_cancellations.nlargest(1))

month
2    0.053705
Name: cancelled, dtype: float64


In [28]:
origin_flight_cancellations = flights.groupby('origin')['cancelled'].mean()
print(origin_flight_cancellations.nsmallest(1))

origin
JFK    0.01977
Name: cancelled, dtype: float64


9. Arrival delay questions:
    a. Which date has the largest average arrival delay (arr_delay)? (Use the format MM/DD - for example, if the answer is Jan 9, use "01/09" in the answer file)
    b. What is the average from (a.) in minutes?

In [62]:
# single largest delay, not the average
# flights[flights['arr_delay'] == flights['arr_delay'].max()][['month', 'day', 'arr_delay']]
avg_arr_delay_by_date = flights.groupby(['month', 'day'])['arr_delay'].mean().reset_index()
max_avg_delay_row = avg_arr_delay_by_date.loc[avg_arr_delay_by_date['arr_delay'].idxmax()]
max_avg_delay_row

month         3.000000
day           8.000000
arr_delay    85.862155
Name: 66, dtype: float64

10. Each plane in the dataset has a unique tailnum.
    a. What is the tailnum of the most common plane in the dataset?
    b. How many times is the plane from (a.) in the dataset?
    c. What is the average distance that the plane from (a.) traveled (in miles)?
    d. What is the most common route (origin to destination) traveled for the plane from (a.)? (Use the format "ORG-DST", for example "JFK-SLC")

In [36]:
flights['tailnum'].value_counts().head(1)

tailnum
N725MQ    575
Name: count, dtype: int64

In [37]:
flights[flights['tailnum'] == 'N725MQ']['distance'].mean()

558.6052173913043

In [40]:
filtered_flights = flights[flights['tailnum'] == 'N725MQ'].copy()
filtered_flights['route'] = filtered_flights['origin'] + '-' + filtered_flights['dest']
filtered_flights['route'].value_counts().head(1)

route
LGA-RDU    173
Name: count, dtype: int64

Questions 11-16 will use one or more other datasets in the Flights folder, including possibly flights.csv

In [42]:
planes = pd.read_csv('planes.csv')
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
weather = pd.read_csv('weather.csv')

11. How many planes have a missing year of manufacture?

In [43]:
planes['year'].isnull().sum()

70

12.  a. Of the planes in the flights, what is the tail number of the oldest plane?
     b. How old was the plane from part (a) in 2013 in years?

In [48]:
planes = planes.rename(columns={'year': 'make_year'})
flights_and_planes = pd.merge(flights, planes, on='tailnum', how='inner')
oldest_plane_row = flights_and_planes.loc[flights_and_planes['make_year'].idxmin()]
oldest_plane_tailnum = oldest_plane_row['tailnum']
print(oldest_plane_tailnum)
print(2013-oldest_plane_row['make_year'])

N381AA
57.0


13. Create a DataFrame with 3 columns: "carrier_name", "carrier", "avg_plane_age".
"carrier_name" is the name of the carrier (i.e., "Delta Air Lines Inc.")
"carrier" is the two letter carrier abbreviation (i.e., "DL")
"avg_plane_age" is the average age of the planes in the flights data by carrier
Sort your the DataFrame by "avg_plane_age" so that the airline with the youngest average plane age is listed first.
Ensure the index goes from 0-15 after you've sorted
Save the final DataFrame as csv file called "avg_carrier_age.csv" and include index=None so the index column is not saved as part of the csv
Include the avg_carrier_age.csv in your GitHub repository
On the answers.txt file, report the age, carrier, and carrier_name of the first two rows.
(a) Row 1 age
(b) Row 1 carrier
(c) Row 1 carrier_name
(d) Row 2 age
(e) Row 2 carrier
(f) Row 2 carrier_name

In [80]:
airlines = airlines.rename(columns={'name': 'carrier_name'})

flights_planes_and_airlines = pd.merge(flights_and_planes, airlines, on='carrier', how='inner').drop_duplicates() # new df with all necessary info
flights_planes_and_airlines['plane_age'] = 2013 - flights_planes_and_airlines['make_year'] # find all plane ages
flights_planes_and_airlines = flights_planes_and_airlines.drop_duplicates(subset='tailnum')
avg_carrier_age = flights_planes_and_airlines.groupby(['carrier', 'carrier_name'])['plane_age'].mean().reset_index() # find average ages
avg_carrier_age = avg_carrier_age.rename(columns={'plane_age': 'avg_plane_age'}) # rename column
avg_carrier_age = avg_carrier_age.sort_values(by='avg_plane_age').reset_index(drop=True) # sort by ages
avg_carrier_age.to_csv('avg_carrier_age.csv', index=False) # send to csv

avg_carrier_age.head(2) # get age, carrier, and carrier_name of the first two rows.

,carrier,carrier_name,avg_plane_age
0,HA,Hawaiian Airlines Inc.,1.230769
1,VX,Virgin America,4.288462


14. What is the correlation coefficient between the number of seats on a plane and the average distance it traveled?

In [56]:
avg_distance_by_plane = flights.groupby('tailnum')['distance'].mean().reset_index()
avg_distance_by_plane.rename(columns={'distance': 'avg_distance'}, inplace=True)

merged_df = pd.merge(avg_distance_by_plane, planes[['tailnum', 'seats']], on='tailnum', how='inner')
merged_df['seats'].corr(merged_df['avg_distance'])

0.545113435221816

15. How many days had precipitation in the New York area in 2013? (That is, how many days had non-zero precipitation at at least one of the airports in the dataset during at least one of the hours.)

In [108]:
total_precip_per_days = weather.groupby(['month', 'day'])['precip'].sum().reset_index()
precip_days_above_zero = total_precip_per_days[total_precip_per_days['precip'] > 0].reset_index(drop=True)
num_days_with_precip = precip_days_above_zero[['month', 'day']].drop_duplicates().shape[0]
num_days_with_precip

141

16. Using only days with precipitation greater than zero, what is the correlation between total precipitation and average minutes of departure delay?
    That is, using only the days with precipitation (that you found in problem 15):
        Compute the total precipitation for the day (across hours and airports)
        Compute the average departure delay in minutes for the day (across hours and airports)
        Compute the correlation between total precipitation and average departure delay

In [111]:
avg_delay = flights.groupby(['month', 'day'])['dep_delay'].mean().reset_index()
avg_delay.rename(columns={'dep_delay': 'avg_dep_delay'}, inplace=True)
precip_delay_merged = pd.merge(precip_days_above_zero, avg_delay, on=['month', 'day'], how='inner')

correlation = precip_delay_merged['precip'].corr(precip_delay_merged['avg_dep_delay'])
print(correlation)

0.2916299956212592
